# Getting Started with NovoMD

This notebook demonstrates the basic usage of the NovoMD API for molecular dynamics and computational chemistry.

## Prerequisites

1. NovoMD server running (locally or remote)
2. API key configured

```bash
# Start NovoMD with Docker
docker run -d -p 8010:8010 -e NOVOMD_API_KEY="your-api-key" ghcr.io/quantnexusai/novomd:latest
```

In [ ]:
import requests
import json

# Configuration
BASE_URL = "http://localhost:8010"
API_KEY = "your-api-key"  # Replace with your actual API key

headers = {
    "Content-Type": "application/json",
    "X-API-Key": API_KEY
}

## 1. Health Check

First, let's verify the server is running:

In [ ]:
response = requests.get(f"{BASE_URL}/health")
print(json.dumps(response.json(), indent=2))

## 2. Check Service Status

View available capabilities and supported force fields:

In [ ]:
response = requests.get(f"{BASE_URL}/status", headers=headers)
print(json.dumps(response.json(), indent=2))

## 3. Convert SMILES to OpenMD Format

Let's convert a simple molecule (ethanol) from SMILES to OpenMD format:

In [ ]:
# Ethanol molecule
payload = {
    "smiles": "CCO",
    "force_field": "AMBER",
    "optimize_3d": True,
    "add_hydrogens": True,
    "box_size": 30.0
}

response = requests.post(
    f"{BASE_URL}/smiles-to-omd",
    headers=headers,
    json=payload
)

result = response.json()
print(f"Success: {result['success']}")
print(f"\nMetadata keys: {list(result['metadata'].keys())}")

## 4. Explore Molecular Properties

NovoMD calculates 32+ molecular properties from the 3D structure:

In [ ]:
metadata = result['metadata']

# Basic info
print("=== Basic Information ===")
print(f"SMILES: {metadata.get('smiles')}")
print(f"Molecular Weight: {metadata.get('molecular_weight')} Da")
print(f"Number of Atoms: {metadata.get('num_atoms')}")
print(f"Number of Bonds: {metadata.get('num_bonds')}")

# Geometry properties
print("\n=== Geometry Properties ===")
print(f"Radius of Gyration: {metadata.get('radius_of_gyration')} Å")
print(f"Asphericity: {metadata.get('asphericity')}")
print(f"Eccentricity: {metadata.get('eccentricity')}")
print(f"Span (max distance): {metadata.get('span_r')} Å")

# Electrostatic properties
print("\n=== Electrostatic Properties ===")
print(f"Dipole Moment: {metadata.get('dipole_moment')} D")
print(f"Total Charge: {metadata.get('total_charge')}")
print(f"Max Partial Charge: {metadata.get('max_partial_charge')}")
print(f"Min Partial Charge: {metadata.get('min_partial_charge')}")

# Surface/Volume
print("\n=== Surface/Volume Properties ===")
print(f"SASA: {metadata.get('sasa')} Å²")
print(f"Molecular Volume: {metadata.get('molecular_volume')} Å³")
print(f"Globularity: {metadata.get('globularity')}")

## 5. View Generated Files

The API returns both PDB and OpenMD format files:

In [ ]:
# PDB content (first 20 lines)
print("=== PDB Content (preview) ===")
pdb_lines = result['pdb_content'].split('\n')[:20]
print('\n'.join(pdb_lines))

In [ ]:
# OpenMD content (first 30 lines)
print("=== OpenMD Content (preview) ===")
omd_lines = result['omd_content'].split('\n')[:30]
print('\n'.join(omd_lines))

## 6. Try Different Molecules

Let's try some common drug molecules:

In [ ]:
molecules = {
    "Aspirin": "CC(=O)OC1=CC=CC=C1C(=O)O",
    "Caffeine": "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",
    "Ibuprofen": "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",
    "Acetaminophen": "CC(=O)NC1=CC=C(C=C1)O"
}

for name, smiles in molecules.items():
    response = requests.post(
        f"{BASE_URL}/smiles-to-omd",
        headers=headers,
        json={"smiles": smiles, "force_field": "AMBER"}
    )
    result = response.json()
    
    if result['success']:
        m = result['metadata']
        print(f"{name}:")
        print(f"  MW: {m.get('molecular_weight', 'N/A')} Da")
        print(f"  Atoms: {m.get('num_atoms_with_h', 'N/A')}")
        print(f"  Radius of Gyration: {m.get('radius_of_gyration', 'N/A')} Å")
        print(f"  SASA: {m.get('sasa', 'N/A')} Å²")
        print()

## Next Steps

- Check out `02_molecular_properties.ipynb` for detailed property analysis
- See `03_visualization.ipynb` for 3D molecular visualization
- View `04_batch_processing.ipynb` for processing multiple molecules